# BlazingSQL vs. Apache Spark 

Below we have one of our popular workloads running with [BlazingSQL](https://blazingsql.com/) + [RAPIDS AI](https://rapids.ai) and then running the entire ETL phase again, only this time with Apache Spark + PySpark.

In this notebook, we will cover: 
- How to read and query csv files with cuDF and BlazingSQL.
- How BlazingSQL compares against Apache Spark (analyzing over 20M records).

#### BlazingSQL install check
The next cell checks that you have BlazingSQL installed, and offers to install it if not (making sure the notebook will run as expected).

In [1]:
import sys 
# point import path notebooks-contrib/utils
sys.path.append('../../../utils/')
from sql_check import bsql_start
# check that BlazingSQL is installed
bsql_start()

"You've got BlazingSQL set up perfectly! Let's get started with SQL in RAPIDS AI!"

## Create BlazingContext
You can think of the BlazingContext much like a Spark Context, this is where information such as FileSystems you have registered and Tables you have created will be stored. 

In [2]:
import cudf
from blazingsql import BlazingContext

bc = BlazingContext()

BlazingContext ready


### Load & Query Table
First, we need to download the netflow data (21,526,138 records) from AWS. 

In [3]:
import os
import urllib.request

data_dir = '../../../data/blazingsql/'
if not os.path.exists(data_dir):
    print('creating blazingsql directory')
    os.system('mkdir ../../../data/blazingsql/'')

In [4]:
# save nf-chunk2 to data folder, may take a few minutes to download
base_url = 'https://blazingsql-colab.s3.amazonaws.com/netflow_data/'
fn = 'nf-chunk2.csv'
if not os.path.isfile(data_dir+fn):
    print(f'Downloading {base_url+fn} to {data_dir+fn}')
    urllib.request.urlretrieve(base_url+fn, data_dir+fn)

## BlazingSQL + cuDF 
Data in hand, we can test the preformance of cuDF and BlazingSQL on this dataset. 

In [5]:
%%time
# Load CSVs into GPU DataFrames (GDF)
netflow_gdf = cudf.read_csv(data_dir+fn)

CPU times: user 2.01 s, sys: 749 ms, total: 2.76 s
Wall time: 2.75 s


In [6]:
%%time
# Create BlazingSQL table from GDF - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

CPU times: user 3.61 ms, sys: 0 ns, total: 3.61 ms
Wall time: 2.53 ms


In [9]:
%%time
# define the query
query = '''
        select
            a.firstSeenSrcIp as source,
            a.firstSeenDestIp as destination,
            count(a.firstSeenDestPort) as targetPorts,
            sum(a.firstSeenSrcTotalBytes) as bytesOut,
            sum(a.firstSeenDestTotalBytes) as bytesIn,
            sum(a.durationSeconds) as durationSeconds,
            min(parsedDate) as firstFlowDate,
            max(parsedDate) as lastFlowDate,
            count(*) as attemptCount
        from 
            netflow a
        group by
            a.firstSeenSrcIp,
            a.firstSeenDestIp
        '''

# query the table (returns cudf dataframe)
result_gdf = bc.sql(query)

CPU times: user 260 ms, sys: 6.59 ms, total: 266 ms
Wall time: 79.6 ms


In [10]:
# how's it look?
result_gdf.head(10)

,source,destination,targetPorts,bytesOut,bytesIn,durationSeconds,firstFlowDate,lastFlowDate,attemptCount
0,172.10.1.234,10.0.0.5,104,47287,64750,18,2013-04-03 06:53:55,2013-04-03 15:11:07,104
1,172.30.1.70,10.0.0.9,79,34894,47870,62,2013-04-03 06:55:15,2013-04-03 12:12:49,79
2,172.30.2.125,10.0.0.9,69,30701,41558,341,2013-04-03 06:50:50,2013-04-03 12:12:37,69
3,172.30.1.56,172.0.0.1,25,3330,3240,67,2013-04-03 01:59:09,2013-04-03 22:05:39,25
4,172.10.1.106,10.199.250.2,40,66638,2863884,24,2013-04-03 07:19:02,2013-04-03 10:12:35,40
5,172.10.1.81,239.255.255.250,1,525,0,6,2013-04-03 06:36:27,2013-04-03 06:36:27,1
6,172.10.1.179,10.1.0.76,70,32252,43677,31,2013-04-03 06:48:44,2013-04-03 15:15:56,70
7,172.20.1.53,10.0.0.7,67,30526,42344,3,2013-04-03 06:53:22,2013-04-03 11:22:04,67
8,172.30.1.73,10.0.0.11,81,36312,50578,82,2013-04-03 06:47:57,2013-04-03 12:10:00,81
9,10.7.5.5,172.20.1.95,2,5723116,134056,59,2013-04-03 10:11:31,2013-04-03 10:59:14,2


## Apache Spark
The cell below installs Apache Spark ([PySpark](https://spark.apache.org/docs/latest/api/python/index.html)).

In [11]:
# installs Spark (2.4.4 Nov 2019)
!pip install pyspark

     |################################| 215.7MB 77.0MB/s eta 0:00:011
     |################################| 204kB 72.4MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=7879a54a037a812709763c4abf7d3d85b5b9b9f8ac6278785942767dc8032f54
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


#### PyBlazing vs PySpark
With everything installed we can launch a SparkSession and see how BlazingSQL stacks up.

In [12]:
%%time
#I copied this cell's snippet from another Google Colab by Luca Canali here: https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb

from pyspark.sql import SparkSession

# Create Spark Session
# This example uses a local cluster, you can modify master to use  YARN or K8S if available 
# This example downloads sparkMeasure 0.13 for scala 2_11 from maven central

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("PySpark Netflow Benchmark code") \
        .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.11:0.13")  \
        .getOrCreate()

CPU times: user 73.4 ms, sys: 27.2 ms, total: 101 ms
Wall time: 6.59 s


### Load & Query Table

In [17]:
%%time
# load CSV into Spark
netflow_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(data_dir+fn)

CPU times: user 20.4 ms, sys: 33.4 ms, total: 53.8 ms
Wall time: 48.8 s


In [18]:
%%time
# create table for querying
netflow_df.createOrReplaceTempView('netflow')

CPU times: user 1.87 ms, sys: 0 ns, total: 1.87 ms
Wall time: 28.6 ms


In [19]:
%%time
# define the same query run tested on blazingsql above
query = '''
        SELECT
            a.firstSeenSrcIp as source,
            a.firstSeenDestIp as destination,
            count(a.firstSeenDestPort) as targetPorts,
            SUM(a.firstSeenSrcTotalBytes) as bytesOut,
            SUM(a.firstSeenDestTotalBytes) as bytesIn,
            SUM(a.durationSeconds) as durationSeconds,
            MIN(parsedDate) as firstFlowDate,
            MAX(parsedDate) as lastFlowDate,
            COUNT(*) as attemptCount
        FROM
            netflow a
        GROUP BY
            a.firstSeenSrcIp,
            a.firstSeenDestIp
        '''

# query with Spark
edges_df = spark.sql(query)

# set/display results
edges_df.show(5)

+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
|      source|    destination|targetPorts|bytesOut|bytesIn|durationSeconds|      firstFlowDate|       lastFlowDate|attemptCount|
+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
| 172.10.1.13|239.255.255.250|         15|    2975|      0|              6|2013-04-03 06:36:19|2013-04-03 06:36:27|          15|
|172.30.1.204|239.255.255.250|          8|    1750|      0|              6|2013-04-03 06:36:13|2013-04-03 06:36:20|           8|
| 172.30.2.86|      172.0.0.1|          1|     540|      0|              2|2013-04-03 06:36:09|2013-04-03 06:36:09|           1|
|172.30.1.246|      172.0.0.1|         29|    2610|   2610|              0|2013-04-03 00:26:46|2013-04-03 23:06:00|          29|
| 172.30.1.51|239.255.255.250|         16|    3850|      0|             18|2013-04-03 06:35:22|20